In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
np.random.seed(42)

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [25]:
fashion = np.concatenate([x_train, x_test], axis = 0)
fashion = np.expand_dims(fashion, axis = -1)

fashion = fashion.astype("float32") / 255

fashion.shape

(70000, 28, 28, 1)

# Creating the GAN Model

## Creating the Discriminator Model

In [5]:
def create_discriminator(in_shape = (28, 28, 1)):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', input_shape = in_shape))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Conv2D(64, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Conv2D(64, (5, 5), strides = (1, 1), padding = 'same'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    # compiling the model
    optimize = keras.optimizers.Adam(lr = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = optimize, metrics = ['accuracy'])
    
    return model

## Define the Discriminator model

In [6]:
discriminator = create_discriminator()
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          1

## Creating the Generator Model

In [19]:
def create_generator(latent_dimension):
    model = keras.Sequential()
    
    # 7x7
    n_nodes = 128 * 7 * 7
    model.add(keras.layers.Dense(n_nodes, input_dim = latent_dimension))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Reshape((7, 7, 128)))
    
    # 14x14
    model.add(keras.layers.Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    
    model.add(keras.layers.Conv2DTranspose(128, (1, 1), strides = (1, 1), padding = 'same'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    
    # 28x28
    model.add(keras.layers.Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    
    model.add(keras.layers.Conv2D(1, (7, 7), activation = 'sigmoid', padding = 'same'))
    
    return model

## Defining the Generator Model

In [21]:
latent_dimension = 100
generator = create_generator(latent_dimension)
generator.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 6272)              633472    
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 6272)              0         
_________________________________________________________________
reshape_6 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_18 (Conv2DT (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_19 (Conv2DT (None, 14, 14, 128)       16512     
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 14, 14, 128)      

In [23]:
def create_gan(discriminator_model, generator_model):
    discriminator_model.trainable = False
    
    gan_model = keras.Sequential()
    gan_model.add(generator_model)
    gan_model.add(discriminator_model)
    
    optimize = keras.optimizers.Adam(lr = 0.0002, beta_1 = 0.5)
    gan_model.compile(loss = 'binary_crossentropy', optimizer = optimize)
    
    return gan_model

In [24]:
gan_model = create_gan(discriminator, generator)
gan_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 28, 28, 1)         1180801   
_________________________________________________________________
sequential (Sequential)      (None, 1)                 143169    
Total params: 1,323,970
Trainable params: 1,180,801
Non-trainable params: 143,169
_________________________________________________________________


In [39]:
def generate_real_samples(dataset, n_samples):
    rand_x = np.random.randint(0, dataset.shape[0], n_samples)
    
    x = dataset[rand_x]
    y = np.ones((n_samples, 1))
    
    return x, y

In [40]:
def generate_latent_points(latent_dimension, n_samples):
    x_input = np.random.randn(latent_dimension * n_samples)
    x_input = x_input.reshape(n_samples, latent_dimension)
    
    return x_input

def generate_fake_samples(generator_model, latent_dimension, n_samples):
    x_input = generate_latent_points(latent_dimension, n_samples)
    x = generator_model.predict(x_input)
    y = np.zeros((n_samples, 1))
    
    return x, y

In [41]:
def summarize_performance(epoch, generator_model, discriminator_model, dataset, latent_dimension, n_samples):
    x_real, y_real = generate_real_samples(dataset, n_samples)
    _, accuracy_real = discriminator_model.evaluate(x_real, y_real, verbose = 0)
    
    x_fake, y_fake = generate_fake_samples(generator_model, latent_dimension, n_samples)
    _, accuracy_fake = discriminator_model.evaluate(x_fake, y_fake, verbose = 0)
    
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (accuracy_real * 100, accuracy_fake * 100))
    

In [44]:
def train(generator_model, discriminator_model, gan_model, dataset, latent_dimension, n_epochs, n_batch):
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            x_real, y_real = generate_real_samples(dataset, half_batch)
            x_fake, y_fake = generate_fake_samples(generator_model, latent_dimension, half_batch)
            
            x, y = np.vstack((x_real, x_fake)), np.vstack((y_real, y_fake))
            
            d_loss, _ = discriminator_model.train_on_batch(x, y)
            
            x_gan = generate_latent_points(latent_dimension, n_batch)
            y_gan = np.ones((n_batch, 1))
            
            gan_loss = gan_model.train_on_batch(x_gan, y_gan)
            
        print('>%d, %d/%d, d_loss = %.3f, gan_loss = %.3f' % (i + 1, j + 1, batch_per_epoch, d_loss, gan_loss))
    
    summarize_performance(i, generator_model, discriminator_model, dataset, latent_dimension, n_epochs)
    
    return generator_model

In [45]:
latent_dimension = 100
trained_generator = train(generator, discriminator, gan_model, fashion, latent_dimension, 20, 256)

>1, 273/273, d_loss = 0.665, gan_loss = 0.635
>2, 273/273, d_loss = 0.679, gan_loss = 0.783


KeyboardInterrupt: 

# Display a plot of generated images

In [ ]:
def display_plot(examples, n):
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap = 'gray_r')
    plt.show()

In [ ]:
latent_points = generate_latent_points(100, 25)
# generate images
X = trained_generator.predict(latent_points)
# plot the result
display_plot(X, 5)